In [42]:
import fitz  # PyMuPDF
import re
import json

In [43]:
def extract_questions_and_options(text):
    questions = []

    # Padrão para corresponder às perguntas e suas opções
    pattern = r"(QUESTÃO \d+\..+?)(?=\nQUESTÃO \d+\.|\Z)"
    matches = re.finditer(pattern, text, re.DOTALL)

    for match in matches:
        question_block = match.group(1)

        # Separar a pergunta das opções
        question_parts = re.split(r"\n(?=[A-D]\.)", question_block)
        question_text = question_parts[0].replace('\n', ' ')  # Substituir quebras de linha por espaço
        options = question_parts[1:]

        # Estruturando a pergunta e opções
        question_dict = {
            "question": question_text.strip(),
            "options": [opt.strip() for opt in options]
        }
        questions.append(question_dict)

    return questions

In [44]:
# Função para limpar e corrigir o texto extraído
def clean_text(text):
    # Substituir caracteres problemáticos
    text = text.replace('', 'fi')
    text = text.replace('×', 'x')

    # Adicione aqui outras substituições conforme necessário

    return text

In [45]:
def main(pdf_file_path):
    # Abrir o arquivo PDF
    doc = fitz.open(pdf_file_path)

    # Extrair e limpar texto de cada página
    cleaned_text = []
    for page in doc:
        page_text = page.get_text()
        # Limpar e corrigir o texto da página
        cleaned_page_text = clean_text(page_text)
        # Usar expressão regular para remover o rodapé
        footer_pattern = r"Medway - EINSTEIN - SP - 2023"
        footer_pattern2 = r"Páginas \d{1,2}/36"
        cleaned_page_text_1 = re.sub(footer_pattern, '', cleaned_page_text)
        cleaned_page_text_2 = re.sub(footer_pattern2, '', cleaned_page_text_1)
        cleaned_text.append(cleaned_page_text_2)

    # Fechar o documento
    doc.close()

    # Concatenar todas as linhas limpas em uma única string para processamento
    cleaned_text_combined = '\n'.join(cleaned_text)

    # Extrair perguntas e suas opções
    extracted_questions = extract_questions_and_options(cleaned_text_combined)

    # Converter a lista de perguntas e opções em formato JSON
    extracted_questions_json = json.dumps(extracted_questions, indent=2, ensure_ascii=False)

    # Salvar o JSON em um arquivo
    with open('output.json', 'w', encoding='utf-8') as f:
        f.write(extracted_questions_json)


In [46]:
pdf_file_path = './../pdfs/EINSTEIN 2023 - Objetiva.pdf'
result = main(pdf_file_path)
print(result)

None
